In [ ]:
import numpy as np
from time import time, sleep
from torch import nn, optim
import torch.nn.functional as F
import torch
import random
from random import randrange, randint
import math
from IPython.display import clear_output
import copy
from collections import deque

In [ ]:
# functions

def drawBoard(board): 
    print( " ",  board[6], " | ", board[7], " | ", board[8])
    print("-----------------")
    print( " ", board[3], " | ", board[4], " | ", board[5])
    print("-----------------")
    print( " ", board[0], " | ", board[1], " | ", board[2])

def checkVertical(positions):
    if(  (positions[6]==positions[3]==positions[0]==2)
      or (positions[7]==positions[4]==positions[1]==2)
      or (positions[8]==positions[5]==positions[2]==2)):
        return 2
    if(  (positions[6]==positions[3]==positions[0]==1)
      or (positions[7]==positions[4]==positions[1]==1)
      or (positions[8]==positions[5]==positions[2]==1)):
        return 1
    return 0

def checkHorizontal(positions):
    if(  (positions[6]==positions[7]==positions[8]==2)
      or (positions[3]==positions[4]==positions[5]==2)
      or (positions[0]==positions[1]==positions[2]==2)):
        return 2
    if(  (positions[6]==positions[7]==positions[8]==1)
      or (positions[3]==positions[4]==positions[5]==1)
      or (positions[0]==positions[1]==positions[2]==1)):
        return 1
    return 0

def checkDiagonal(positions):
    if(  (positions[6]==positions[4]==positions[2]==2)
      or (positions[0]==positions[4]==positions[8]==2)):
        return 2
    if(  (positions[6]==positions[4]==positions[2]==1)
      or (positions[0]==positions[4]==positions[8]==1)):
        return 1
    return 0

def checkAnyWin(board):
    if(checkVertical(board)==1 or checkHorizontal(board)==1 or checkDiagonal(board)==1 ):
#         print("Player 1 wins")
        return 1
    if(checkVertical(board)==2 or checkHorizontal(board)==2 or checkDiagonal(board)==2 ):
#         print("Player 2 wins")
        return 2
    return 0    

def play(board, remain, action, turn):
    if(len(remain)>0):
        if(action in remain):
            board[action]=turn
            remain.remove(action)            
        else:
            return -10
        
    return checkAnyWin(board)

def win_checker(board, remain, player):
    b = board
    if(player==1):
        for i in remain:
            b[i]=1
            x = checkAnyWin(b)
            b[i]=0            
            if(x==1):
                return i
    else:
        for i in remain:
            b[i]=2
            x = checkAnyWin(b)
            b[i]=0            
            if(x==2):
                return i
    return -1      

def win_blocker(board, remain, player):
    b = board
    if(player==1):
        for i in remain:
            b[i]=2
            x = checkAnyWin(b)
            b[i]=0            
            if(x==2):
                return i
    else:
        for i in remain:
            b[i]=1
            x = checkAnyWin(b)
            b[i]=0            
            if(x==1):
                return i
    return -1     

def oneHot(board):
    return torch.tensor( [(F.one_hot(torch.tensor(board), num_classes=3) ).tolist()], dtype=torch.float32).flatten()

def makeOneHot(player , pos):
    x = math.floor(random.random()*9)
    a = [0.0]*9
    a[pos] = player
    return a

In [ ]:
# define models

# dequeue to store recent experiences 
p1=deque(maxlen=2500)
p2=deque(maxlen=2500)

training1 = nn.Sequential(
    nn.Linear(27, 40),
    nn.ReLU(),
    nn.Linear(40, 9),
    nn.Softmax())
target1 = copy.deepcopy(training1)
mse1 = nn.MSELoss()
opt1 = optim.Adam(training1.parameters(), lr=0.001)
training2 = nn.Sequential(
    nn.Linear(27, 40),
    nn.ReLU(),
    nn.Linear(40, 9),
    nn.Softmax())
target2 = copy.deepcopy(training2)
mse2 = nn.MSELoss()
opt2 = optim.Adam(training2.parameters(), lr=0.001)


In [ ]:
# training cell 

discount_factor = 0.95
error = 0 # count number or errors made during experience collection
a1w=0
a2w=0
ties=0

#training method:

for t in range(20):
    # t = 0 to 5 means first 96,000 games with these values of epsilon decay 
    if(t==0):
        epsilon = 1
        decay = 0.9999
        firstRandom = False
        print("done ", t)
    # t = 6,7 means first 32,000 games with these values of epsilon decay 
    elif(t==6):
        epsilon = 0.1
        decay = 1
        firstRandom = False      
        print("done ", t)  
    elif(t==8):
        epsilon = 0.5
        decay = 1
        firstRandom = False
        print("done ", t)
    elif(t==10):
        epsilon = 0.3
        decay = 1
        firstRandom = True
        print("done ", t)
    elif(t==13):
        epsilon = 0.3
        decay = 0.9999
        firstRandom = True
        print("done ", t)
    elif(t==18):
        epsilon = 0.2
        decay = 0.9999
        firstRandom = False
        print("done ", t)

    for k in range(16000):
        # experience collection
        board = [0,0,0,0,0,0,0,0,0]
        remain = [0, 1, 2, 3, 4, 5, 6, 7, 8]
        done = False
        first = math.floor(random.random()*2)
        for i in range(9):
            reward=0
            a=[None]*5
            a[0]=(board[:])

            if(i==8):
                done=True

            # who plays first. 0 = training1 plays first, 1 = training2 plays first
            # this if is for gameplay of agent1
            if(i%2==first): 
                # this if structure will provide first move of agent random if firstRandom = True
                if(i==0 and firstRandom):  
                    action = random.choice(remain)
                    actions_random = makeOneHot(1, action)
                    for j in range(9):
                        actions[j] = actions_random[j]
                    a[1] = actions[:]
                
                else:
                    actions = training1(oneHot(board))
                    wc = win_checker(board, remain, 1)
                    wb = win_blocker(board, remain, 1)

                    if(wc!=-1): # win checking function output 
                        action = wc
                        actions_random = makeOneHot(1, wc)
                        for j in range(9):
                            actions[j] = actions_random[j]
                        a[1] = actions[:]
                    elif(wb!=-1): # opponent win blocking function output 
                        action = wb
                        actions_random = makeOneHot(1, wb)
                        for j in range(9):
                            actions[j] = actions_random[j]
                        a[1] = actions[:]
                    else:
                        if(random.uniform(0,1)<epsilon): # epsilon random strategy
                            action = math.floor(random.random()*9)
                            actions_random = makeOneHot(1, action)
                            for j in range(9):
                                actions[j] = actions_random[j]
                            a[1] = actions[:]
                        else: # output of the actual neural network of the agent 
                            actions = training1(oneHot(board))
                            action = actions.argmax().item()
                            a[1] = actions[:]

                x = play(board, remain, action, 1)
            
            # everything same as agent1 but for agent2
            else:
                if(i==0 and firstRandom):
                    action = random.choice(remain)
                    actions_random = makeOneHot(2, action)
                    for j in range(9):
                        actions[j] = actions_random[j]
                    a[1] = actions[:]
                else:
                    actions = training2(oneHot(board))
                    wc = win_checker(board, remain, 2)
                    wb = win_blocker(board, remain, 2)

                    if(wc!=-1): # win checking function output 
                        action = wc
                        actions_random = makeOneHot(2, wc)
                        for j in range(9):
                            actions[j] = actions_random[j]
                        a[1] = actions[:]
                    elif(wb!=-1): # opponent win blocking function output 
                        action = wb
                        actions_random = makeOneHot(2, wb)
                        for j in range(9):
                            actions[j] = actions_random[j]
                        a[1] = actions[:]
                    else:

                        if(random.uniform(0,1)<epsilon): # epsilon random strategy
                            action = math.floor(random.random()*9)
                            actions_random = makeOneHot(2, action)
                            for j in range(9):
                                actions[j] = actions_random[j]
                            a[1] = actions[:]
                        else: # output of the actual neural network of the agent 
                            actions = training2(oneHot(board))   
                            action = actions.argmax().item()
                            a[1] = actions[:]

                x = play(board, remain, action, 2)

            a[2]=(board[:])

            if(x==0):   # if played on not played position
                reward += 1 # reward for not playing on already played position
                a[3]=reward
                a[4]=False
                if(i==8):
#                     print("Tie")
                    ties+=1
                    a[3]=5 # reward of tie for the agent that played last 
                    a[4]=True   
                    if(i%2==first): # reward for tie for the other opponent 
                        p2[len(p2)-1][3]=5
                    else:
                        p1[len(p1)-1][3]=5

            if(x==1 or x==2): # when one of them wins
                if(x==1):
                    a1w+=1
                if(x==2):
                    a2w+=1
                done=True
                reward += 20
                a[3]=reward
                a[4]=True
#                 print("Winner is ",x)
                if(i%2==first):
                    p1.append(a)
                else:
                    p2.append(a)
    #             print(a)
                break        
            if(x==-10): # if played on already played position
#                 print("Gameplay error, played on already played position")
                error+=1
                done=True
                a[3]=-20
                a[4]=True
                if(i%2==first):
                    p1.append(a)
                else:
                    p2.append(a)
                break
                
            # append the experience tuple to the respective dequeue
            if(i%2==first):
                p1.append(a)
            else:
                p2.append(a)
        epsilon*=decay

        # training of models from the experiences selected randomly from dequeue
        if(k%600==0 and k!=0):
            print("Errors in batch of 600 games = ",error)
            print("Agent 1 wins =", a1w)
            print("Agent 2 wins =", a2w)
            print("Ties = ", ties)
            a1w=0
            a2w=0
            ties=0
            error=0
            print("learning")
            samples1 = random.sample(p1, 500)
            samples2 = random.sample(p2, 500)
            for i in samples1: # i is a experience tuple (s a s' r a)
                if(not i[4]):
                    next_q_value = target1(oneHot(i[2])).max().item()
                    y = i[3] + discount_factor*next_q_value
                else:
                    y = i[3]

                t = training1(oneHot(i[0]))

                y_target = [0]*9
                for j in range(9):
                    if(i[0][j]!=0):
                        y_target[j]=-10
                y_target[i[1].argmax().item()] = y 
                y_target = torch.tensor(y_target, dtype=torch.float32)

                loss = mse1(t , y_target)
                opt1.zero_grad()
                loss.backward()
                opt1.step()  

            for i in samples2: # i is a experience tuple (s a s' r a)
                if(not i[4]):
                    next_q_value = target2(oneHot(i[2])).max().item()
                    y = i[3] + discount_factor*next_q_value
                else:
                    y = i[3]

                t = training2(oneHot(i[0]))

                y_target = [0]*9
                for j in range(9):
                    if(i[0][j]!=0):
                        y_target[j]=-10 # make all played positions -10 reward 
                y_target[i[1].argmax().item()] = y # actual reward for the played position
                y_target = torch.tensor(y_target, dtype=torch.float32)

                loss = mse2(t , y_target)
                opt2.zero_grad()
                loss.backward()
                opt2.step()    
        # every 1200 games, the target network will be updated  
        # to weights of the training network 
        if(k%1200==0):
            target1 = copy.deepcopy(training1)
            target2 = copy.deepcopy(training2)

In [ ]:
# watch 2 agents play against each other 


board=[0]*9
remain = [0,1,2,3,4,5,6,7,8]
first = math.floor(random.random()*2)
for i in range(9):
    
    print("")
    print("")
     # who plays first. 
     # 0 = Agent 1 plays first
     # 1 = Agent 2 plays first
        
    if(i%2==first):
        
        #if else structure to make first move random
        
        if(i==0): 
            action = random.choice(remain)
        else:
            actions = training1(oneHot(board))
            action = actions.argmax().item()

        #if you don't want first move random in playing 
        # uncomment these 2 lines and comment if..else
        
#         actions = training1(oneHot(board))
#         action = actions.argmax().item()
        
        x = play(board, remain, action, 1)
        if(x==-10):
            print("Agent 1 lost tried to play on ", action)
            break
            break

    else:
        #if else structure to make first move random
        
        if(i==0):
            action = random.choice(remain)
        else:
            actions = training2(oneHot(board))
            action = actions.argmax().item()

        # if you don't want first move random in playing 
        # uncomment these 2 lines and comment if..else
        
#         actions = training2(oneHot(board))
#         action = actions.argmax().item()
        
        x = play(board, remain, action, 2)    
        if(x==-10):
            print("Agent 2 lost tried to play on ", action)
            break
    drawBoard(board)
    sleep(1)
    clear_output()

if(x==0):
    print("Tie")
elif(x==1):
    print("Agent 1 won")
elif(x==2):
    print("Agent 2 won")
drawBoard(board)
print("")
print("")

In [ ]:
# play against one of the agents 

board=[0]*9
remain = [0,1,2,3,4,5,6,7,8]
first = 1
for i in range(9):
    drawBoard(board)
    
     # who plays first. 
     # 0 = Agent 1 plays first
     # 1 = Agent 2 plays first
        
    if(i%2==first): 
        action = int(input()) - 1
        x = play(board, remain, action, 2)
        if(x==-10):
            print("player 1 lost")
            break
        elif(x==1 or x==2):
            print("p1  won")
            break
        

    else:
        actions = training1(oneHot(board))
        action = actions.argmax().item()
        x = play(board, remain, action, 1)    
        if(x==-10):
            print("agent 1 lost", "tried to play on ", action+1)
            break
        elif(x==1 or x==2):
            print("p2 won")
            break
    sleep(1)
    clear_output()
    sleep(0.1)
    
drawBoard(board)

In [ ]:
# save the 4 networks ,
# to name files differently replace the names in inverted commas 

torch.save(training1, "training1.x")
torch.save(target1, "target1.x")
torch.save(training2, "training2.x")
torch.save(target2, "target2.x")

In [ ]:
# initialize the networks from the files stored

training1 = nn.Sequential(
    nn.Linear(27, 40),
    nn.ReLU(),
    nn.Linear(40, 9),
    nn.Softmax() )
training1 = torch.load("training1.x", weights_only=False)

target1 = nn.Sequential(
    nn.Linear(27, 40),
    nn.ReLU(),
    nn.Linear(40, 9),
    nn.Softmax() )
target1 = torch.load("target1.x", weights_only=False)

training2 = nn.Sequential(
    nn.Linear(27, 40),
    nn.ReLU(),
    nn.Linear(40, 9),
    nn.Softmax() )
training2 = torch.load("training2.x", weights_only=False)

target2 = nn.Sequential(
    nn.Linear(27, 40),
    nn.ReLU(),
    nn.Linear(40, 9),
    nn.Softmax() )
target2 = torch.load("target2.x", weights_only=False)